![title](UCN ICI.png)

 #  <font color='#7E9BC0'> Data Analysis con Python - Solución del Problema</font>

<div class="alert alert-info"> </h4> **Material preparado para el Master en Ingeniería Civil Industrial | Universidad Católica del Norte | Campus Antofagasta | Curso Electivo - Predictive Analytics.**
</h4> </div>

## <font color='#2d89ef'> Problema 1: </font>

G&A produce y distribuye productos para el cuidado personal. Los productos se fabrican en 6 plantas diferentes y luego se envían a diferentes centros de distribución en todo el país. Los gerentes de algunos centros de distribución han estado informando la existencia de desabastecimientos en los últimos meses.

Se le encargó analizar los datos disponibles de inventario para cada SKU en cada centro de distribución para cuantificar qué tan grande es el problema. Para realizar esta tarea, recibió dos archivos de Excel que contienen cuatro columnas que se describen a continuación:

- date: fecha
- dc: id del centro de distribución
- sku: id del SKU
- inventory: inventario on-hand (en cajas)


Para completar su análisis, realice las siguientes tareas:


1. Lea los 2 archivos de Excel (dc_inventory (1) .xlsx y dc_inventory (2) .xlsx)
2. Concatene los 2 conjuntos de datos de inventario para crear un único conjunto de datos. El DataFrame resultante debe contener 118,650 filas.
3. Ordene los valores por DC, SKU y Fecha en orden ascendente
4. Identifique cuántos centros de distribución únicos se incluyen en el conjunto de datos
5. Identifique cuántas SKU únicas se incluyen en el conjunto de datos
6. Agregue una nueva columna llamada 'oos' para identificar las existencias agotadas. Si el inventario disponible> 0, entonces el valor de esta columna debe ser 0, de lo contrario 1. **Hint: use la función lambda**
7. Cree un nuevo DataFrame que calcule el número de días con falta de existencias por DC y SKU. Las columnas de este nuevo DataFrame deben ser 'dc', 'sku' y 'oos_days'. El número de días con existencias agotadas (oos_days) debe ser la suma de 'oos' para cada SKU en ecah DC. **Hint: use groupby**
8. En el nuevo DataFrame, elimine esas filas con 350 días de desabastecimiento
9. Identifique el DC y SKU con el mayor número de días con desabastecimientos
10. Guarde el DataFrame resultante como un archivo de Excel

**Pregunta nº 1:**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from os import getcwd
from os import listdir
from pandas import read_excel

path=getcwd()
files = listdir(path)
files

['dc_shipments.xlsx',
 'dc_inventory 1.xlsx',
 'dc_inventory 2.xlsx',
 'UCN ICI.png',
 '.ipynb_checkpoints',
 'Data Analytics - Problemas Propuestos.ipynb',
 'Data Analytics - Solución Problemas Propuestos.ipynb']

In [3]:
excel_files=[x for x in files if x.endswith(".xlsx") ]
excel_files


['dc_shipments.xlsx', 'dc_inventory 1.xlsx', 'dc_inventory 2.xlsx']

In [4]:
excel_file_1=excel_files[1]
excel_file_2=excel_files[2]

In [5]:
df_1=pd.read_excel(excel_file_1)
df_2=pd.read_excel(excel_file_2)

**Pregunta nº 2:**

In [6]:
frames = [df_1, df_2]
df=pd.concat(frames)
display(df.head())
df.shape

,date,dc,sku,inventory
0,2016-07-01,220,1001,0
1,2016-07-02,220,1001,0
2,2016-07-18,540,1002,0
3,2016-07-17,540,1002,0
4,2016-07-19,540,1002,0


(118650, 4)

**Pregunta nº 3:**

In [7]:
result= df.sort_values(['dc', 'sku','date'], ascending=[1, 1,1])
result.head()

,date,dc,sku,inventory
8786,2016-07-01,100,1003,0
8787,2016-07-02,100,1003,0
40970,2016-07-03,100,1003,0
40969,2016-07-04,100,1003,0
40965,2016-07-05,100,1003,0


**Pregunta nº 4:**

In [8]:
df_1 = df.groupby('dc')['dc'].nunique()
print('Hay '+str(len(df_1))+'centros de distribución diferentes')
df_1.head()

Hay 45centros de distribución diferentes


dc
100    1
110    1
120    1
130    1
140    1
Name: dc, dtype: int64

**Pegunta nº 5:**

In [9]:
df_2 = df.groupby('sku')['sku'].nunique()
print('Hay '+str(len(df_2))+' SKU')
df_2.head()

Hay 121 SKU


sku
1001    1
1002    1
1003    1
1004    1
1005    1
Name: sku, dtype: int64

**Pregunta nº 6:**

In [10]:
df['oos']=df.apply(lambda row: 0 if row['inventory']>0 else 1,axis=1)
display(df.head())
df.shape

,date,dc,sku,inventory,oos
0,2016-07-01,220,1001,0,1
1,2016-07-02,220,1001,0,1
2,2016-07-18,540,1002,0,1
3,2016-07-17,540,1002,0,1
4,2016-07-19,540,1002,0,1


(118650, 5)

**Pregunta nº 7:**

In [11]:
DataFrame=df['oos'].groupby([df['sku'],df['dc']]).sum().reset_index(name='oos_days')
DataFrame.head()

,sku,dc,oos_days
0,1001,220,350
1,1001,330,350
2,1001,380,350
3,1002,260,246
4,1002,540,350


**Pregunta nº 8:**

In [12]:
indexValue = DataFrame[ DataFrame['oos_days'] == 350 ].index
DataFrame.drop(indexValue , inplace=True)
DataFrame.head()

,sku,dc,oos_days
3,1002,260,246
6,1003,240,8
7,1003,280,6
8,1003,360,10
9,1003,400,3


**Pregunta nº 9:**

In [13]:
x=DataFrame['oos_days'].idxmax()
print('El SKU con más dias con stock out es el '+str(DataFrame.iloc[x][2])+ ' ubicado en el DC '+ str(DataFrame.iloc[x][1]))

El SKU con más dias con stock out es el 330 ubicado en el DC 220


**Pregunta nº 10:**

In [14]:
DataFrame.to_excel('Resulting.xlsx')

## <font color='#2d89ef'> Problema 2: </font>

G&A sospecha que las existencias perdidas causan desabastecimientos. Para probar esta hipótesis, se le encargó analizar los datos de envío durante el período de 1 año. Le entregaron un archivo Excel que contiene la información detallada a continuación:

- date: fecha
- plant: Nombre de la planta de Origen
- dc: ID del Centro de Distribución de destino
- sku: ID del SKU 
- ordered: numero of caja ordenada
- delivered: numero de cajas entregadas


Para completar su análisis, realice las siguientes tareas:



1. Lea el archivo de Excel que contiene datos de envío de plantas a DC (dc_shipments.xlsx)
2. Ordene los valores por DC, SKU, Planta y Fecha en orden ascendente
3. Cree una nueva columna (perdida) para calcular el número de cajas perdidos (ordenados - entregados)
4. Cree un nuevo DataFrame que calcule el número de cajas perdidas por Plant. **Hint: use groupby**
5. Identifique las 3 plantas principales con el mayor número de cajas perdidas
6. Cree un nuevo DataFrame que calcule el número de cajas perdidas por DC. **Hint: use groupby**
7. Identifique los 5 principales DC con el mayor número de cajas perdidas.
8. Cree un nuevo DataFrame que calcule el número de cajas perdidas por SKU. **Hint: use groupby**
9. Identifique los 5 SKU principales con el mayor número de cajas perdidas.

**Pregunta nº 1:**

In [15]:
dataframe=pd.read_excel(excel_files[0])
dataframe.head()

,date,plant,dc,sku,ordered,delivered
0,2016-10-02,West,400,1080,88.0,88
1,2016-11-07,West,280,1003,36.0,36
2,2016-07-02,South west,300,1085,48.0,48
3,2016-09-22,Midwest,320,1013,3.0,3
4,2016-09-07,South west,270,1086,10.0,10


**Pregunta nº 2:**

In [16]:
dataframe= dataframe.sort_values(['dc', 'sku','plant'], ascending=[1, 1,1])
dataframe.head()

,date,plant,dc,sku,ordered,delivered
51,2016-08-20,South east,130,1006,92.0,92
79,2017-04-04,South east,130,1006,115.0,115
286,2016-07-30,South east,130,1006,20.0,20
482,2017-03-15,South east,130,1006,115.0,115
1277,2016-08-05,South east,130,1006,20.0,20


**Pregunta nº 3:**

In [17]:
dataframe['missed']=dataframe['ordered']-dataframe['delivered']
dataframe.head()

,date,plant,dc,sku,ordered,delivered,missed
51,2016-08-20,South east,130,1006,92.0,92,0.0
79,2017-04-04,South east,130,1006,115.0,115,0.0
286,2016-07-30,South east,130,1006,20.0,20,0.0
482,2017-03-15,South east,130,1006,115.0,115,0.0
1277,2016-08-05,South east,130,1006,20.0,20,0.0


**Pregunta nº 4:**

In [18]:
NewDataFrame=dataframe.groupby(['plant']).sum()
NewDataFrame

,dc,sku,ordered,delivered,missed
plant,,,,,
Mid-atlantic,951850,2857763,54628.0,54350,278.0
Midwest,1100230,3488458,107739.0,107001,738.0
New England,1190490,4211361,94890.0,93999,891.0
South east,1442120,4402765,109338.0,107481,1857.0
South west,818470,2619942,84765.0,82842,1923.0
West,1908220,5596980,124365.0,122289,2076.0


**Pregunta nº 5:**

In [19]:
NewDataFrame= NewDataFrame.sort_values(['missed'], ascending=0)
for n in range(3):
    print('**********')
    print('Planta : '+str(NewDataFrame.index.values[n]))
    print('**********')
    print(NewDataFrame.iloc[n])
    print('-------------------------------')

**********
Planta : West
**********
dc           1908220.0
sku          5596980.0
ordered       124365.0
delivered     122289.0
missed          2076.0
Name: West, dtype: float64
-------------------------------
**********
Planta : South west
**********
dc            818470.0
sku          2619942.0
ordered        84765.0
delivered      82842.0
missed          1923.0
Name: South west, dtype: float64
-------------------------------
**********
Planta : South east
**********
dc           1442120.0
sku          4402765.0
ordered       109338.0
delivered     107481.0
missed          1857.0
Name: South east, dtype: float64
-------------------------------


**Pregunta nº 6:**

In [20]:
NewDataFrame=dataframe.groupby(['dc']).sum()
NewDataFrame= NewDataFrame.sort_values(['missed'], ascending=0)
NewDataFrame.head(5)

,sku,ordered,delivered,missed
dc,,,,
450,379921,19311.0,18702,609.0
150,290074,15284.0,14676,608.0
480,668278,25569.0,24980,589.0
270,773739,31575.0,31032,543.0
300,818752,22836.0,22311,525.0


**Pregunta nº 7:**

In [21]:
NewDataFrame= NewDataFrame.sort_values('missed',ascending=0)
for n in range(5):
    print('**********')
    print('DC Nº '+str(NewDataFrame.index.values[n]))
    print('**********')
    print('El número de cajas perdidad por el DC es: '+str(NewDataFrame.iloc[n][3]))
    print("")
    print('-------------------------------')

**********
DC Nº 450
**********
El número de cajas perdidad por el DC es: 609.0

-------------------------------
**********
DC Nº 150
**********
El número de cajas perdidad por el DC es: 608.0

-------------------------------
**********
DC Nº 480
**********
El número de cajas perdidad por el DC es: 589.0

-------------------------------
**********
DC Nº 270
**********
El número de cajas perdidad por el DC es: 543.0

-------------------------------
**********
DC Nº 300
**********
El número de cajas perdidad por el DC es: 525.0

-------------------------------


**Pregunta nº 8:**

In [22]:
NewDataFrame=dataframe.groupby(['sku']).sum()
NewDataFrame.head()

,dc,ordered,delivered,missed
sku,,,,
1001,123810,6825.0,6752,73.0
1002,33420,2670.0,2594,76.0
1003,119400,11907.0,11789,118.0
1004,86440,21550.0,21024,526.0
1005,570,9.0,9,0.0


**Pregunta nº 9:**

In [23]:
NewDataFrame= NewDataFrame.sort_values('missed',ascending=0)
for n in range(5):
    print('**********')
    print('SKU Nº '+str(NewDataFrame.index.values[n]))
    print('**********')
    print('El número de cajas perdidas por SKU es: '+str(NewDataFrame.iloc[n][3]))
    print("")
    print('-------------------------------')

**********
SKU Nº 1004
**********
El número de cajas perdidas por SKU es: 526.0

-------------------------------
**********
SKU Nº 1052
**********
El número de cajas perdidas por SKU es: 479.0

-------------------------------
**********
SKU Nº 1064
**********
El número de cajas perdidas por SKU es: 473.0

-------------------------------
**********
SKU Nº 1077
**********
El número de cajas perdidas por SKU es: 470.0

-------------------------------
**********
SKU Nº 1086
**********
El número de cajas perdidas por SKU es: 384.0

-------------------------------
